# Imports data from tissue analyser and performs some further analysis

In [1]:
!pip install xarray

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
# load external packages

from curses import meta
import os
import os.path
import argparse

import logging
import warnings
import time
from copy import deepcopy
from typing import Tuple

import numpy as np
import pandas as pd
import xarray as xr

import math
from scipy.optimize import curve_fit
from scipy.interpolate import griddata

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import seaborn as sns

import yaml
from PIL import Image
from PIL.TiffTags import TAGS

In [15]:
# load segmentation scripts

from import_data_function import import_data
from utils import *

In [4]:
# initialize logger

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s :: %(levelname)s :: %(message)s')
log = logging.getLogger("DataImport")
log.setLevel(logging.DEBUG)

log.info("Initialized logger")

2023-03-30 11:33:22,350 :: INFO :: Initialized logger


In [10]:
cells, bonds = import_data(config_file="sample-data/import_config.yml", log=log)

2023-03-30 11:38:46,696 :: INFO :: Loading data at /home/julian/tissue_segmentation/bp-segmentation
2023-03-30 11:38:46,751 :: INFO :: Using channel sum_px_intensity_cells_ch1 for HCA criterion
2023-03-30 11:38:46,753 :: INFO :: Detected 22.37 perc. HC
2023-03-30 11:38:46,754 :: INFO :: HC density per file: 
2023-03-30 11:38:46,796 :: INFO :: Setting column aligned_with_PD_axis: `True' (from default).
2023-03-30 11:38:46,797 :: INFO :: Loaded data at /home/julian/tissue_segmentation/bp-segmentation in 0.10s.
2023-03-30 11:38:46,799 :: INFO :: ---------
2023-03-30 11:38:46,799 :: INFO :: Important Info:
2023-03-30 11:38:46,800 :: INFO :: 
2023-03-30 11:38:46,800 :: INFO :: HCs have an average HCA criterion of:
2023-03-30 11:38:46,817 :: INFO :: Gathered 608 cells (incl. 136 HC) at stages ['E14'] and positions ['75I']. 1 files: 
 ['E14_75I_6_0.tif']
2023-03-30 11:38:46,820 :: INFO :: --------------------
2023-03-30 11:38:46,820 :: INFO :: Retrieving meta-data
2023-03-30 11:38:46,821 :: I

filename
E14_75I_6_0.tif    0.223684
dtype: float64
file_id
0    255.0
Name: HCA_criterion, dtype: float64


2023-03-30 11:38:50,022 :: INFO :: Number of HC-HC junctions: 
filename         type
E14_75I_6_0.tif  HH      3
Name: type, dtype: int64
2023-03-30 11:38:50,022 :: INFO :: Assigned neighbourhood in 3.18 s.
2023-03-30 11:38:50,023 :: INFO :: ---
2023-03-30 11:38:50,023 :: DEBUG :: Normalizing length scales ...
2023-03-30 11:38:50,090 :: INFO :: Normalized length scales in 0.07 s.
2023-03-30 11:38:50,090 :: DEBUG :: Calculating hexatic order ..
2023-03-30 11:38:52,367 :: INFO :: Calculated hexatic order in 2.28 s.
2023-03-30 11:38:52,367 :: INFO :: Cell tracking skipped.
2023-03-30 11:38:52,368 :: DEBUG :: Importing cilia ..
2023-03-30 11:38:52,368 :: DEBUG :: Parsing cilia import args: 
{'path': 'sample-data/cilia-data', 'HC_only': True, 'max_radius': 1.2, 'import_kwargs': {'sep': ',', 'header': 0, 'pixel_per_micron': [1.0, 1.0]}, 'sort_values': {'by': ['rho'], 'ascending': [True]}}
2023-03-30 11:38:53,307 :: INFO :: Assigned 54 cilia for 136 HC in 0.94s.
2023-03-30 11:38:53,307 :: INFO